In [ ]:
import os
import datetime as dt
from datetime import date, datetime
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from IPython.display import display
import matplotlib.dates as mdates
from matplotlib import style
import seaborn as sns
import statsmodels.api as sm
from pylab import mpl
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tools.sm_exceptions import InterpolationWarning
import scipy.stats as scs
import scipy.optimize as sco
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from collections import OrderedDict
import warnings
from pathlib import Path

In [ ]:
# Valori di default dei grafici
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 100

# INIZIALIZZAZIONE DELLE VARIABILI
# Dizionario dei Ticker
tcks = {
    "Tech" : ["AMZN", "GOOGL"],
    "Auto" : ["TSLA", "F"],
    "Med" : ["PFE", "NVAX"]
}

# Array dei settori
sects = list(tcks.keys())

# Array dei Ticker
tickers = []
for sett in tcks:
        for tck in tcks[sett]:
            tickers.append(tck)

# Nomi completi
nomi = {
    "AMZN" : "Amazon",
    "GOOGL" : "Google",
    "TSLA" : "Tesla",
    "F" : "Ford",
    "PFE" : "Pfizer",
    "NVAX" : "Novavax",
    "Tech" : "Tecnologico",
    "Auto" : "Automotive",
    "Med" : "Medico-Farmaceutico"
}

# Date
day = ['2014-05-31', '2024-05-31']

def init(frq=None):
    asts = {}
    # Crea la directory 'dati' se non esiste
    Path('dati').mkdir(parents=True, exist_ok=True)
    
    for sett in tcks:
        for tck in tcks[sett]:
            file = "dati/" + tck + ".csv"
            if Path(file).is_file():
                # Carica i dati dal file CSV
                asts[tck] = pd.read_csv(file)
                asts[tck].index = pd.to_datetime(asts[tck].Date)
            else:
                # Scarica i dati usando yfinance
                asts[tck] = yf.download(tck, start=day[0], end=day[1])
                asts[tck].to_csv(file)
            if frq == "M" or frq == "W":
                # Raggruppa i dati per frequenza mensile o settimanale
                asts[tck] = asts[tck].resample(frq).mean()
    return asts

assets = init("ME")

adj = {}
adj["Complete"]  = pd.DataFrame()
for sett in tcks:
    adj[sett]  = pd.DataFrame()
    for tck in tcks[sett]:
        ## Adjusted Close per ogni asset
        adj[tck] = pd.DataFrame(assets[tck]["Adj Close"])
        adj[tck].columns = [tck]
        ## Adjusted Close per ogni settore
        adj[sett] = pd.concat([adj[sett], adj[tck]], axis = 1)
    ## Adjusted Close completo
    adj["Complete"] = pd.concat([adj["Complete"], adj[sett]], axis = 1)

# Crea la directory 'img/Adj Close' se non esiste
#Path("img/Adj Close").mkdir(parents=True, exist_ok=True)

adj["Complete"].plot()
plt.legend()
plt.title("Adjusted Close")

# Save
#plt.savefig("img/Adj Close/all.png")

In [ ]:
#Visto che i dati che stiamo usando dati scaricati sono stati presi usando lo stesso metodo con stesse date di inizio e di fine possiamo quindi presumere che forma di questi siano la medesiama per ogni asset.
#Per fare questo tipo di analisi prenderemo in considerazione quindi un solo asset, prendiamo in considerazione Amazon
amzn = assets["AMZN"]
print(amzn.shape)
amzn.axes
amzn.head()

#Per ogni giorno (per ogni riga quindi) rappresentiamo 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close' dove
type(amzn["High"])
amzn["High"].dtype

type(amzn["Low"])
amzn["Low"].dtype

type(amzn["Open"])
amzn["Open"].dtype

type(amzn["Close"])
amzn["Close"].dtype

type(amzn["Volume"])
amzn["Volume"].dtype

type(amzn["Adj Close"])
amzn["Adj Close"].dtype

#Stampiamo ora per ciascuno dei titoli
assets["AMZN"].head()
adj["AMZN"].plot(grid=True, title = 'Adjusted Close Price di Amazon')

assets["GOOGL"].head()
adj["GOOGL"].plot(grid=True, title = 'Adjusted Close Price di Google')

assets["TSLA"].head()
adj["TSLA"].plot(grid=True, title = 'Adjusted Close Price di Tesla')

assets["F"].head()
adj["F"].plot(grid=True, title = 'Adjusted Close Price di Ford')

assets["PFE"].head()
adj["PFE"].plot(grid=True, title = 'Adjusted Close Price di Pfizer')

assets["NVAX"].head()
adj["NVAX"].plot(grid=True, title = 'Adjusted Close Price di Novavax')


In [ ]:
#Calcolo dei ritorni con rispettivi grafici per ogni asset
rndS = {} # rendimenti semplici
rndS["Complete"]  = pd.DataFrame()
for sett in tcks:
    rndS[sett]  = pd.DataFrame()
    for tck in tcks[sett]:
        rndS[tck] = adj[tck].pct_change()
        #rndS[tck] = np.log(adj[tck]/adj[tck].shift(1))
        rndS[tck] = rndS[tck].dropna()
        rndS[sett] = pd.concat([rndS[sett], rndS[tck]], axis = 1)
    rndS["Complete"] = pd.concat([rndS["Complete"], rndS[sett]], axis = 1)
rndS["Complete"].head()

plt.rcParams['figure.figsize'] = [15, 18]
fig, axs = plt.subplots(3, 2)
j = 0
for sett in tcks:
    i = 0
    for tck in tcks[sett]:
        axs[j, i].plot(rndS[tck])
        axs[j, i].set_title("Rendimenti semplici di " + nomi[tck])
        i += 1
    j += 1

plt.suptitle("Rendimenti Semplici", fontsize = 40)
# Save
#plt.savefig("img/Semplici_Assets.png")

In [ ]:
# Controllo dei rendimenti semplici per singolo settore
plt.rcParams['figure.figsize'] = [15, 18]
fig, axs = plt.subplots(3, 2)
j = 0
for sett in tcks:
    i = 0
    for tck in tcks[sett]:
        axs[j, i].plot(rndS[tck])
        axs[j, i].set_title("Rendimenti semplici di " + nomi[tck])
        i += 1
    j += 1

plt.suptitle("Rendimenti Semplici", fontsize = 40)
# Save
#plt.savefig("img/Semplici_Assets.png")

In [ ]:
# Controllo dei rendimenti composti per i diversi titoli
rndC = {} # Rendimenti Composti
rndC["Complete"]  = pd.DataFrame()
for sett in tcks:
    rndC[sett]  = pd.DataFrame()
    for tck in tcks[sett]:
        rndC[tck] = (rndS[tck] + 1).cumprod()
        rndC[sett] = pd.concat([rndC[sett], rndC[tck]], axis = 1)
    rndC["Complete"] = pd.concat([rndC["Complete"], rndC[sett]], axis = 1)
    rndC[sett] = pd.concat([rndC[tcks[sett][0]], rndC[tcks[sett][1]]], axis = 1)
rndC["Complete"].head()


plt.rcParams['figure.figsize'] = [15, 18]
fig, axs = plt.subplots(3, 2)
j = 0
for sett in tcks:
    i = 0
    for tck in tcks[sett]:
        axs[j, i].plot(rndC[tck])
        axs[j, i].set_title("Rendimenti composti di " + nomi[tck])
        i += 1
    j += 1
plt.suptitle("Rendimenti Composti", fontsize = 40)
#Save
#plt.savefig("img/Composti_assets.png")

In [ ]:
# Controllo dei rendimenti composti per i settori 
fig, axs = plt.subplots(3, 1)
j = 0
for sett in tcks:
    axs[j].plot(rndC[sett])
    axs[j].legend(tcks[sett])
    axs[j].set_title("Rendimenti composti del settore " + nomi[sett])
    j += 1
#Save
#plt.savefig("img/Composti_settori.png")

In [ ]:
#Funzioni per la Correlazione dei titoli
def correlazione (correlation_index):
    if correlation_index == 0:
        stringa = " non c'è nessuna correlazione"
    else:
        sg = ""  # Segno
        sig = "" # Significatività
        if correlation_index > 0:
            sg = "positiva"
        else:
            sg = "negativa"
        if abs(correlation_index) <= 0.3:
            sig = " poco significativa"
        elif abs(correlation_index) >= 0.7:
            sig = " molto significativa"
        stringa = " c'è una correlazione di " + str(correlation_index) + " ovvero una correlazione " + sg + sig
    return stringa

def min_corr (df): # passo il dataframe contente le sole correlazioni
    ind = df.keys()
    m = (ind[0], ind[1])
    for i in ind:
        for j in ind:
            if (j != i) & (df[i][j] < df[m[0]][m[1]]):
                m = (i, j)
    return m

def max_corr (df): # passo il dataframe contente le sole correlazioni
    ind = df.keys()
    m = (ind[0], ind[1])
    for i in ind:
        for j in ind:
            if (j != i) & (df[i][j] > df[m[0]][m[1]]):
                m = (i, j)
    return m

def sectMax (df):
    max = sects[0]
    for sett in sects:
        if (df[tcks[sett][0]][tcks[sett][1]] > df[tcks[max][0]][tcks[max][1]]):
            max = sett
    return max

def sectMin (df):
    min = sects[0]
    for sett in sects:
        if (df[tcks[sett][0]][tcks[sett][1]] < df[tcks[min][0]][tcks[min][1]]):
            min = sett
    return min

# Stampa dei risultati dell'applicazione delle funzioni di correlazione
for sett in tcks:
    corr = rndS[sett].corr()[tcks[sett][0]][tcks[sett][1]]
    print ("Tra " + nomi[tcks[sett][0]] + " e " + nomi[tcks[sett][1]] + correlazione(corr))

In [ ]:
#Istogramma dei rendimenti
plt.rcParams['figure.figsize'] = [15, 18]
i = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(i)
        rndS[tck].plot.density(ax = plt.subplot(i))
        plt.hist(rndS[tck], density = True)
        plt.title("Distribuzione dei rendimenti di " + nomi[tck])
        i += 1

# Save
#plt.savefig("img/Distribuzione dei rendimenti.png")

In [ ]:
#Grafici diagnostici - Metodo per reare grafici diagnostici a 4 sezioni (istogramma, kernel density, boxplot, qq-plot) per ciascuna serie di rendimenti
for tck in assets:
    plt.figure(figsize=(16, 9))

    # Istogramma
    plt.subplot(221)
    sns.histplot(data = rndS[tck], stat = "density")
    plt.title("Distribuzione dei rendimenti di " + nomi[tck])

    # kernel density
    plt.subplot(222)
    sns.histplot(data = rndS[tck], stat = "density", kde = True)
    plt.title("Kernel density di " + nomi[tck])
    
    # Boxplot 
    plt.subplot(223)
    #plt.boxplot(rndS[tck], sym = "x")
    sns.boxplot(data = rndS[tck], orient = "v", width = 0.4)
    plt.title("Boxplot di " + nomi[tck])
        
    # qq-plot
    sm.qqplot(rndS[tck], ax = plt.subplot(224))
    plt.title("QQ-Plot di " + nomi[tck])
    
    # Titolo del grafico 
    plt.suptitle(nomi[tck])
    
    # Save
    #plt.savefig("img/4Grafici/distr_rend_" + tck +".png")

In [ ]:
# Definizione delle statistiche
st_i = ["Media", "Var", "SD", "Skew", "Kur"]
st_univ = pd.DataFrame(columns=st_i, index=assets.keys())

# Calcolo delle statistiche descrittive
for tck in assets:
    st_univ.loc[tck, "Media"] = rndS[tck].mean().values[0] if isinstance(rndS[tck].mean(), pd.Series) else rndS[tck].mean()
    st_univ.loc[tck, "Var"] = rndS[tck].var().values[0] if isinstance(rndS[tck].var(), pd.Series) else rndS[tck].var()
    st_univ.loc[tck, "SD"] = rndS[tck].std().values[0] if isinstance(rndS[tck].std(), pd.Series) else rndS[tck].std()
    st_univ.loc[tck, "Skew"] = rndS[tck].skew().values[0] if isinstance(rndS[tck].skew(), pd.Series) else rndS[tck].skew()
    st_univ.loc[tck, "Kur"] = rndS[tck].kurtosis().values[0] if isinstance(rndS[tck].kurtosis(), pd.Series) else rndS[tck].kurtosis()

# Visualizza le statistiche
print(st_univ)

# Creo un DataFrame con le descrizioni dei rendimenti ottenute per ogni asset
desc = rndS["Complete"].describe()
print(desc)

# Funzioni per ottenere i valori minimi e massimi
def max_valInd(df):
    tcks = list(assets.keys())
    max_tck = tcks[0]
    for r in tcks:
        if df.loc[r] > df.loc[max_tck]:
            max_tck = r
    return max_tck, df.loc[max_tck]

def min_valInd(df):
    tcks = list(assets.keys())
    min_tck = tcks[0]
    for r in tcks:
        if df.loc[r] < df.loc[min_tck]:
            min_tck = r
    return min_tck, df.loc[min_tck]

def maxAbs_valInd(df):
    tcks = list(assets.keys())
    max_tck = tcks[0]
    for r in tcks:
        if abs(df.loc[r]) > abs(df.loc[max_tck]):
            max_tck = r
    return max_tck, df.loc[max_tck]

def minAbs_valInd(df):
    tcks = list(assets.keys())
    min_tck = tcks[0]
    for r in tcks:
        if abs(df.loc[r]) < abs(df.loc[min_tck]):
            min_tck = r
    return min_tck, df.loc[min_tck]

# Dizionario per la traduzione dei nomi delle statistiche
nomi = {
    "Media": "media",
    "Var": "varianza",
    "SD": "deviazione standard",
    "Kur": "curtosi",
    "Skew": "asimmetria"
}

# Funzione per stampare i valori minimi e massimi delle statistiche
def stampeMinMaxStat(stat):
    max_tck, max_val = max_valInd(st_univ[stat])
    min_tck, min_val = min_valInd(st_univ[stat])
    print(f"L'asset con {nomi[stat]} minore tra tutti è {min_tck} con {min_val}")
    print(f"L'asset con {nomi[stat]} maggiore tra tutti è {max_tck} con {max_val}")

def stampeAbsMinMaxStat(stat):
    max_tck, max_val = maxAbs_valInd(st_univ[stat])
    min_tck, min_val = minAbs_valInd(st_univ[stat])
    print(f"L'asset con {nomi[stat]} assoluta minore tra tutti è {min_tck} con {min_val}")
    print(f"L'asset con {nomi[stat]} assoluta maggiore tra tutti è {max_tck} con {max_val}")

# Funzioni per spiegare il significato di curtosi e asimmetria
def signKur(val):
    if val == 0:
        return "ho una distribuzione uguale alla normale"
    elif val > 0:
        return "gli eventi estremi sono meno frequenti rispetto alla normale"
    else:
        return "gli eventi estremi sono più frequenti rispetto alla normale"

def signSkew(val):
    if val == 0:
        return "ho una distribuzione uguale alla normale"
    elif val > 0:
        return "la curva di frequenza ha una coda più lunga a destra del massimo centrale"
    else:
        return "la curva di frequenza ha una coda più lunga a sinistra del massimo centrale"

In [ ]:
# Come si evolvono nel tempo rendimento e volatilità?
std = {}
for tck in assets:
    stdS = []
    med = []
    for i in range(1, len(rndS[tck].index) + 1):
        # Calcolare deviazione standard e media senza avvisi di deprecazione
        stdS.append(rndS[tck][:i].std())
        med.append(rndS[tck][:i].mean())
    std[tck] = rndS[tck].copy()
    std[tck].rename(columns={tck: "Rendimenti"}, inplace=True)
    std[tck]["std"] = stdS
    std[tck]["mean"] = med

plt.rcParams['figure.figsize'] = [15, 18]
i = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(i)
        plt.plot(std[tck][["std", "mean"]])
        plt.legend(["Deviazione standard Rendimenti", "Media Rendimenti"])
        if tck in nomi:
            plt.title(nomi[tck])
        else:
            plt.title(tck)  # Usa il ticker come titolo se il nome non è trovato
        i += 1
plt.suptitle("Evoluzione rendimenti e volatilità", fontsize=40)
plt.show()
# Save
# plt.savefig("img/Evoluzione rendimenti e volatilita.png")

# Quale azione ha la distribuzione di rendimenti più vicina o lontana dalla normale?
print(st_univ[["Kur", "Skew"]])

st = "Skew"
for tck in assets:
    val = st_univ[st][tck]
    print(f"L'asset {nomi.get(tck, tck)} ha un valore di {nomi[st]} di {val:.2f} vuol dire che {signSkew(val)}")
stampeAbsMinMaxStat(st)
stampeMinMaxStat(st)

st = "Kur"
for tck in assets:
    val = st_univ[st][tck]
    print(f"L'asset {nomi.get(tck, tck)} ha un valore di {nomi[st]} di {val:.2f} vuol dire che {signKur(val)}")
stampeAbsMinMaxStat(st)
stampeMinMaxStat(st)



In [ ]:
# Calcolo della covarianza tra i rendimenti degli asset
cov = rndS["Complete"].cov()
print(cov)

# Stampa le statistiche minime e massime della varianza
stampeMinMaxStat("Var")

def covarianza(cov):
    """Descrive la covarianza tra due variabili"""
    string = " c'è una covarianza di " + f"{cov:.4f}"
    if cov == 0:
        return string + " non c'è nessuna associazione (lineare) tra xi e yi"
    elif cov > 0:
        return string + " i modelli delle variabili corrispondono"
    else:
        return string + " i modelli delle variabili sono sfalsati, cioè quando una è più alta della sua media l'altra è più bassa della sua media"

# `nomi` contenga i nomi per tutti gli asset, se non si trova un nome per un settore, usa il ticker stesso come fallback
for sett in tcks:
    sett_name = nomi.get(sett, sett)  # Usa il ticker se il nome non è presente
    print("-" * 30 + " " + sett_name + " " + "-" * 30)
    # Assumiamo che tcks[sett] contenga solo due asset
    asset1 = tcks[sett][0]
    asset2 = tcks[sett][1]
    cov_value = cov[asset1][asset2]
    print(f"Tra {nomi.get(asset1, asset1)} e {nomi.get(asset2, asset2)}: {covarianza(cov_value)}")

# Trova il settore con la covarianza maggiore e minore
print("Il settore del mio portfolio con la covarianza maggiore è il settore " + nomi.get(sectMax(cov), "Sconosciuto"))
print("Il settore del mio portfolio con la covarianza minore è il settore " + nomi.get(sectMin(cov), "Sconosciuto"))

# Trova la coppia di asset con la covarianza maggiore e minore
mc_max = max_corr(cov)
print(f"La covarianza maggiore tra i vari titoli è tra {nomi.get(mc_max[0], mc_max[0])} e {nomi.get(mc_max[1], mc_max[1])} che è di {cov[mc_max[0]][mc_max[1]]:.4f}")

mc_min = min_corr(cov)
print(f"La covarianza minore tra i vari titoli è tra {nomi.get(mc_min[0], mc_min[0])} e {nomi.get(mc_min[1], mc_min[1])} che è di {cov[mc_min[0]][mc_min[1]]:.4f}")

# Stampa tutte le covarianze
ind = cov.columns  # Ottieni i nomi delle colonne per evitare problemi con i tuoi indici
for i in range(len(ind)):
    for j in range(i):
        asset1 = ind[i]
        asset2 = ind[j]
        cov_value = cov[asset1][asset2]
        print(f"Tra {nomi.get(asset1, asset1)} e {nomi.get(asset2, asset2)}: {covarianza(cov_value)}")


In [ ]:
# Definizione delle funzioni per trovare la coppia di asset con la covarianza più alta e più bassa
def max_corr(cov):
    """Trova la coppia di asset con la covarianza più alta"""
    max_value = -float('inf')
    max_pair = (None, None)
    keys = list(cov.columns)  # Usa cov.columns invece di cov.keys()
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            if cov[keys[i]][keys[j]] > max_value:
                max_value = cov[keys[i]][keys[j]]
                max_pair = (keys[i], keys[j])
    return max_pair

def min_corr(cov):
    """Trova la coppia di asset con la covarianza più bassa"""
    min_value = float('inf')
    min_pair = (None, None)
    keys = list(cov.columns)  # Usa cov.columns invece di cov.keys()
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            if cov[keys[i]][keys[j]] < min_value:
                min_value = cov[keys[i]][keys[j]]
                min_pair = (keys[i], keys[j])
    return min_pair

# Calcolo della covarianza tra i rendimenti degli asset
cov = rndS["Complete"].cov()
print(cov)

# Stampa le statistiche minime e massime della varianza
stampeMinMaxStat("Var")

def covarianza(cov):
    """Descrive la covarianza tra due variabili"""
    string = " c'è una covarianza di " + f"{cov:.4f}"
    if cov == 0:
        return string + " non c'è nessuna associazione (lineare) tra xi e yi"
    elif cov > 0:
        return string + " i modelli delle variabili corrispondono"
    else:
        return string + " i modelli delle variabili sono sfalsati, cioè quando una è più alta della sua media l'altra è più bassa della sua media"

# Assicurati che `nomi` contenga i nomi per tutti gli asset
# Se non trovi un nome per un settore, usa il ticker stesso come fallback
for sett in tcks:
    sett_name = nomi.get(sett, sett)  # Usa il ticker se il nome non è presente
    print("-" * 30 + " " + sett_name + " " + "-" * 30)
    # Assumiamo che tcks[sett] contenga solo due asset
    asset1 = tcks[sett][0]
    asset2 = tcks[sett][1]
    cov_value = cov[asset1][asset2]
    print(f"Tra {nomi.get(asset1, asset1)} e {nomi.get(asset2, asset2)}: {covarianza(cov_value)}")

# Trova il settore con la covarianza maggiore e minore
#print("Il settore del mio portfolio con la covarianza maggiore è il settore " + nomi.get(sectMax(cov), "Sconosciuto"))
#print("Il settore del mio portfolio con la covarianza minore è il settore " + nomi.get(sectMin(cov), "Sconosciuto"))

# Trova la coppia di asset con la covarianza maggiore e minore
mc_max = max_corr(cov)
print(f"La covarianza maggiore tra i vari titoli è tra {nomi.get(mc_max[0], mc_max[0])} e {nomi.get(mc_max[1], mc_max[1])} che è di {cov[mc_max[0]][mc_max[1]]:.4f}")

mc_min = min_corr(cov)
print(f"La covarianza minore tra i vari titoli è tra {nomi.get(mc_min[0], mc_min[0])} e {nomi.get(mc_min[1], mc_min[1])} che è di {cov[mc_min[0]][mc_min[1]]:.4f}")

# Stampa tutte le correlazioni
ind = cov.columns  # Usa cov.columns invece di cov.keys()
for i in range(len(ind)):
    for j in range(i):
        asset1 = ind[i]
        asset2 = ind[j]
        cov_value = cov[asset1][asset2]
        print(f"Tra {nomi.get(asset1, asset1)} e {nomi.get(asset2, asset2)}: {covarianza(cov_value)}")
corr



In [ ]:
# Grafico scatter plot dei rendimenti semplici per le coppie di titoli del settore Tecnologico
plt.figure(figsize=(12, 6))
plt.scatter(rndS["AMZN"], rndS["GOOGL"], alpha=0.7)
plt.title('Rendimenti Semplici: Amazon vs Google (Settore Tecnologico)')
plt.xlabel('Rendimenti Semplici di Amazon')
plt.ylabel('Rendimenti Semplici di Google')
plt.grid(True)

# Grafico scatter plot dei rendimenti semplici per le coppie di titoli del settore Automotive
plt.figure(figsize=(12, 6))
plt.scatter(rndS["TSLA"], rndS["F"], alpha=0.7)
plt.title('Rendimenti Semplici: Tesla vs Ford (Settore Automotive)')
plt.xlabel('Rendimenti Semplici di Tesla')
plt.ylabel('Rendimenti Semplici di Ford')
plt.grid(True)

# Grafico scatter plot dei rendimenti semplici per le coppie di titoli del settore Farmaceutico
plt.figure(figsize=(12, 6))
plt.scatter(rndS["PFE"], rndS["NVAX"], alpha=0.7)
plt.title('Rendimenti Semplici: Pfizer vs Novavax (Settore Farmaceutico)')
plt.xlabel('Rendimenti Semplici di Pfizer')
plt.ylabel('Rendimenti Semplici di Novavax')
plt.grid(True)

In [ ]:
# Grafico scatter plot dei rendimenti composti per le coppie di titoli del settore Tecnologico
plt.figure(figsize=(12, 6))
plt.scatter(rndC["AMZN"], rndC["GOOGL"], alpha=0.7)
plt.title('Rendimenti Composti: Amazon vs Google (Settore Tecnologico)')
plt.xlabel('Rendimenti Composti di Amazon')
plt.ylabel('Rendimenti Composti di Google')
plt.grid(True)

# Grafico scatter plot dei rendimenti composti per le coppie di titoli del settore Automotive
plt.figure(figsize=(12, 6))
plt.scatter(rndC["TSLA"], rndC["F"], alpha=0.7)
plt.title('Rendimenti Composti: Tesla vs Ford (Settore Automotive)')
plt.xlabel('Rendimenti Composti di Tesla')
plt.ylabel('Rendimenti Composti di Ford')
plt.grid(True)

# Grafico scatter plot dei rendimenti composti per le coppie di titoli del settore Farmaceutico
plt.figure(figsize=(12, 6))
plt.scatter(rndC["PFE"], rndC["NVAX"], alpha=0.7)
plt.title('Rendimenti Composti: Pfizer vs Novavax (Settore Farmaceutico)')
plt.xlabel('Rendimenti Composti di Pfizer')
plt.ylabel('Rendimenti Composti di Novavax')
plt.grid(True)


In [ ]:
# Evoluzioni correlazioni Semplici
correlazioniS = {}
for sett in tcks:
    cors = []
    for i in range(0, len(rndS[sett].index)):
        cors.append(rndS[sett][:i].corr().loc[tcks[sett][0]][tcks[sett][1]])
    correlazioniS[sett] = pd.DataFrame(cors, rndS[sett].index, [sett])
    correlazioniS[sett].dropna(inplace=True)

# Evoluzioni correlazioni Composte
correlazioniC = {}
for sett in tcks:
    cors = []
    for i in range(0, len(rndC[sett].index)):
        cors.append(rndC[sett][:i].corr().loc[tcks[sett][0]][tcks[sett][1]])
    correlazioniC[sett] = pd.DataFrame(cors, rndC[sett].index, [sett])
    correlazioniC[sett].dropna(inplace=True)

# Semplici
plt.rcParams['figure.figsize'] = [15, 18]
j = 311
for sett in tcks:
    plt.subplot(j)
    plt.plot(correlazioniS[sett], label=tcks[sett][0] + " - " + tcks[sett][1]+ " (Semplici)", lw=2)
    plt.legend()
    if tcks[sett][0] in nomi and tcks[sett][1] in nomi:
        plt.title(nomi[tcks[sett][0]] + " - " + nomi[tcks[sett][1]])
    j += 1
plt.suptitle("Evoluzione correlazioni Semplici", fontsize=40)

# Save
# plt.savefig("img/Evoluzione_correlazioni_sem.png")

# Composte
plt.rcParams['figure.figsize'] = [15, 18]
j = 311
for sett in tcks:
    plt.subplot(j)
    plt.plot(correlazioniC[sett], label=tcks[sett][0] + " - " + tcks[sett][1]+ " (Composte)", lw=2)
    plt.legend()
    if tcks[sett][0] in nomi and tcks[sett][1] in nomi:
        plt.title(nomi[tcks[sett][0]] + " - " + nomi[tcks[sett][1]])
    j += 1
plt.suptitle("Evoluzione correlazioni", fontsize=40)

# Save
# plt.savefig("img/Evoluzione_correlazioni_Comp.png")

In [ ]:
#MODELLO DI PREVISIONE SVM(Support Vector Model)

# Definizione dei dati di esempio
tcks = {
    "Tech": ["AMZN", "GOOGL"],
    "Auto": ["TSLA", "F"],
    "Med": ["PFE", "NVAX"]
}

nomi = {
    'AMZN': 'Amazon.com Inc.',
    'GOOGL': 'Alphabet Inc.',
    'TSLA': 'Tesla Inc.',
    'F': 'Ford Motor Company',
    'PFE': 'Pfizer Inc.',
    'NVAX': 'Novavax Inc.',
}

# Esempio di dati
dates = pd.date_range(start='2020-01-01', periods=100, freq='M')
adj = {
    'AMZN': pd.Series(range(100), index=dates),
    'GOOGL': pd.Series(range(100), index=dates),
    'TSLA': pd.Series(range(100), index=dates),
    'F': pd.Series(range(100), index=dates),
    'PFE': pd.Series(range(100), index=dates),
    'NVAX': pd.Series(range(100), index=dates),
}

fc_out = 10
tt = {}
lr_prediction = {}
rbf_prediction = {}
lin_prediction = {}
poly_prediction = {}
df_pred = {}# dizionario dei DataFrame delle predizioni 

for tck in assets:
    df_pred[tck] = pd.DataFrame(assets[tck]["Adj Close"])
    df_pred[tck]["Prediction"] = df_pred[tck]["Adj Close"].shift(-fc_out)
    

    # dataset indipendente --> feature
    feature =  np.array(df_pred[tck][["Adj Close"]])
    feature = feature[:-fc_out]

    # dataset dipendente --> target
    target =  np.array(df_pred[tck]['Prediction'])
    target = target[:-fc_out]
    '''
    tt[...][0] --> x_train
    tt[...][1] --> x_test
    tt[...][2] --> y_train
    tt[...][3] --> y_test
    tt[...][4] --> x_forecast
    '''
    tt[tck] = train_test_split(feature, target, test_size = 30)
    tt[tck].append(np.array(df_pred[tck][["Adj Close"]])[-fc_out:])

In [ ]:
#Regressione Lineare 
for tck in assets:
    print ("-"*30)
    print(nomi[tck])
    lr = LinearRegression()
    lr.fit(tt[tck][0], tt[tck][2])
    print('lr confindence: ', lr.score(tt[tck][1], tt[tck][3]))
    lr_prediction[tck] = lr.predict(tt[tck][4])
    print(lr_prediction[tck])

In [ ]:
# Kernel RBF (Radial Basis Function)
for tck in assets:
    print ("-"*30)
    print(nomi[tck])
    svr_rbf = SVR(kernel='rbf', C = 100000, gamma = 0.0001)
    svr_rbf.fit(tt[tck][0], tt[tck][2])
    print('svr_rbf  confindence: ', svr_rbf.score(tt[tck][1], tt[tck][3]))
    rbf_prediction[tck] = svr_rbf.predict(tt[tck][4])
    print(rbf_prediction[tck])

In [ ]:
# Kernel polinomiale
for tck in assets:
    print ("-"*30)
    print(nomi[tck])
    svr_poly = SVR(kernel='poly', C = 1000, degree = 2)
    svr_poly = svr_poly.fit(tt[tck][0], tt[tck][2])
    print('svr_poly confindence: ', svr_poly.score(tt[tck][1], tt[tck][3]))
    poly_prediction[tck] = svr_poly.predict(tt[tck][4])
    print(poly_prediction[tck])

In [ ]:
# Kernel lineare
for tck in assets:
    print ("-"*30)
    print(nomi[tck])
    svr_lin = SVR(kernel='linear', C = 100)
    svr_lin.fit(tt[tck][0], tt[tck][2])
    print('svr_lin confindence: ', svr_lin.score(tt[tck][1], tt[tck][3]))
    lin_prediction[tck] = svr_lin.predict(tt[tck][4])
    print(lin_prediction[tck])

In [ ]:
import matplotlib.pyplot as plt

# STAMPA GRAFICO
plt.figure(figsize=(20, 20))
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        plt.title("SVM - " + nomi[tck])
        
        # Traccia le serie di dati solo se esistono
        if tck in tt:
            plt.plot(tt[tck][4], color='red', label='Effettivo')
        if tck in lr_prediction:
            plt.plot(lr_prediction[tck], color='lightblue', label='Linear Regression')
        if tck in rbf_prediction:
            plt.plot(rbf_prediction[tck], color='lightgreen', label='RBF')
        if tck in poly_prediction:
            plt.plot(poly_prediction[tck], color='purple', label='Polynomial')
        if tck in lin_prediction:
            plt.plot(lin_prediction[tck], color='orange', label='Linear')
        
        plt.legend()
        j += 1

plt.suptitle("Previsioni - SVM", fontsize=40)
# Save
# plt.savefig("img/SVM.png")
plt.show()


In [ ]:
# Costruire una strategia di trading basata su un algoritmo a scelta che segnali l’acquisto o la vendita di un titiolo o indice di borsa e farne il backtesting

# SMA

# Funzione di inizializzazione per ottenere dati
def init(freq):
    # Esempio di ticker, sostituisci con il tuo elenco
    tickers = ["AMZN", "GOOGL", "TSLA", "F", "PFE", "NVAX"]

    data = {}
    for ticker in tickers:
        df = yf.download(ticker, start="2014-01-01", end="2024-01-01")
        if freq == "M" or freq == "W":
            df = df.resample(freq).mean(numeric_only=True)  # Assicurati di considerare solo colonne numeriche
        data[ticker] = df
    return data

# Codice per costruire una strategia di trading basata su medie mobili semplici e farne il backtesting
data = init("W")
SMA1 = 5
SMA2 = 25
strategy = {}

for tck in data:
    df = pd.DataFrame(data[tck]["Adj Close"])

    # Controllo e conversione dei tipi di dati
    df["Adj Close"] = pd.to_numeric(df["Adj Close"], errors='coerce')
    
    strategy[tck] = df
    strategy[tck]["SMA1"] = strategy[tck]["Adj Close"].rolling(SMA1).mean()
    strategy[tck]["SMA2"] = strategy[tck]["Adj Close"].rolling(SMA2).mean()
    strategy[tck].dropna(inplace=True)
    strategy[tck]["Position"] = np.where(strategy[tck]["SMA1"] > strategy[tck]["SMA2"], 1, -1)
    
print(data["AMZN"].tail())

plt.rcParams['figure.figsize'] = [18, 20]
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        strategy[tck].plot(legend=None, secondary_y='Position', ax=plt.subplot(j), title=nomi[tck])
        j += 1
plt.suptitle("Backtesting medie mobili semplici", fontsize=30)

# Save the plot
#plt.savefig("img/backtesting/SMA.png")

In [ ]:
# Costruire una strategia di trading basata su un algoritmo a scelta che segnali l’acquisto o la vendita di un titiolo o indice di borsa e farne il backtesting

# EWM

for tck in data:
    strategy[tck]['Price_yesterday'] = strategy[tck]['Adj Close'].shift(1)
    strategy[tck]['Change'] = strategy[tck]['Adj Close'] / strategy[tck]['Price_yesterday']
    strategy[tck]['EWM1'] = strategy[tck]['Adj Close'].ewm(span=5, adjust=False).mean()
    strategy[tck]['EWM2'] = strategy[tck]['Adj Close'].ewm(span=12, adjust=False).mean()
    strategy[tck]['Invested_EWM'] = np.where(strategy[tck]["EWM1"] > strategy[tck]["EWM2"], 1, -1)

sma = {}
ewm = {}
for tck in data:
    #medie mobili
    sma[tck] = strategy[tck][strategy[tck]['Position'] == 1]
    sma[tck]['Return'] = np.cumprod(sma[tck]['Change'])
    sma[tck]['rtn'] = sma[tck]['Return'].pct_change()

    #Buy and hold
    strategy[tck]['Buy_and_hold'] = np.cumprod(strategy[tck]['Change'])
    strategy[tck]['rtn'] = strategy[tck]['Buy_and_hold'].pct_change()

    #medie mobili esponenziali
    ewm[tck] = strategy[tck][strategy[tck]['Invested_EWM'] == 1]
    ewm[tck]['Return'] = np.cumprod(ewm[tck]['Change'])

plt.rcParams['figure.figsize'] = [18, 20]
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        strategy[tck][['EWM1', 'EWM2', 'Adj Close', 'Invested_EWM']].plot(legend = None, secondary_y = 'Invested_EWM', ax = plt.subplot(j), title = nomi[tck])
        plt.title(nomi[tck])
        j += 1
plt.suptitle("Backtesting medie mobili esponenziali ", fontsize = 30)

# Save
#plt.savefig("img/backtesting/EWM.png")

In [ ]:
# Misurare l’efficienza della strategia rispetto alla detenzione del titolo per tutto il periodo 
# Confronto tra SMA - Buy&Hold - EWM
plt.rcParams['figure.figsize'] = [18, 20]
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        plt.plot(strategy[tck]['Buy_and_hold'], label = 'Buy and hold')
        plt.plot(sma[tck]['Return'], label = 'SMA')
        plt.plot(ewm[tck]['Return'], label ='EWM')
        plt.legend()
        plt.title(nomi[tck])
        j += 1
plt.suptitle("SMA - Buy & hold - EWM", fontsize = 30)

# Save
#plt.savefig("img/backtesting/Rendimenti3.png")

In [ ]:
# Utilizzare variabili di mercato ma non collegate ai prezzi passati del titolo (volume, VIX, andamento dell’indice o variabili non di mercato (Google Trends)

#OBW / OBW_EWM

obv = init("W")

for tck in assets:
    l_obv = []
    l_obv.append(0)
    for i in range (1, len(obv[tck]["Adj Close"])):
        if (obv[tck]["Adj Close"][i] > obv[tck]["Adj Close"][i-1]):
            l_obv.append(l_obv[-1] + obv[tck]["Volume"][i])
        elif (obv[tck]["Adj Close"][i] < obv[tck]["Adj Close"][i-1]):
            l_obv.append(l_obv[-1] - obv[tck]["Volume"][i])
        else:
            l_obv.append(l_obv[-1])
    obv[tck]["OBV"] = l_obv
    obv[tck]["OBV_EWM"] = obv[tck]["OBV"].ewm(span=20).mean()
    obv[tck]['Price_yesterday'] = obv[tck]['Adj Close'].shift(1)
    obv[tck]['Change'] = obv[tck]['Adj Close'] / obv[tck]['Price_yesterday']

obv_rtn = {}

for tck in assets:
    sigBuy = []
    sigSell = []
    flag = -1
    for i in range (0, len(obv[tck]["OBV"])):
        if (obv[tck]["OBV"][i] > obv[tck]["OBV_EWM"][i]) and flag != 1:
            flag = 1
            sigBuy.append(obv[tck]["Adj Close"][i])
            sigSell.append(np.nan)
        elif (obv[tck]["OBV"][i] < obv[tck]["OBV_EWM"][i]) and flag != 0:
            flag = 0
            sigSell.append(obv[tck]["Adj Close"][i])
            sigBuy.append(np.nan)
        else:
            sigSell.append(np.nan)
            sigBuy.append(np.nan)
    obv[tck]["Sig_Sell"] = sigSell
    obv[tck]["Sig_Buy"] = sigBuy
    obv[tck]['Position'] = np.where(obv[tck]["OBV"] > obv[tck]["OBV_EWM"], 1, -1)
    obv_rtn[tck] = obv[tck][obv[tck]['Position'] == 1]
    obv_rtn[tck]['Return'] = np.cumprod(obv[tck]['Change'])

j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        plt.title(nomi[tck])
        plt.plot(obv[tck]["OBV"], label = "OBV")
        plt.plot(obv[tck]["OBV_EWM"], label = "OBV_EWM")
        j += 1
plt.suptitle("OBV / OBV_EWM", fontsize = 40)

# Save
#plt.savefig("img/backtesting/OBV.png")

In [ ]:
# Strategia OBV
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        plt.title(nomi[tck])
        plt.scatter(obv[tck].index, obv[tck]["Sig_Buy"], label = "Sig_Buy", marker = '^', color = "green")
        plt.scatter(obv[tck].index, obv[tck]["Sig_Sell"], label = "Sig_Sell", marker = 'v', color = "red")
        plt.plot(obv[tck]["Adj Close"], label = "Adj Close")
        j += 1
plt.suptitle("OBV strategy", fontsize = 40)

# Save
#plt.savefig("img/backtesting/OBV_strategy1.png")

In [ ]:
# Strategia OBV
plt.rcParams['figure.figsize'] = [18, 20]
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        obv[tck][['Adj Close', 'Position']].plot(legend = None, secondary_y = 'Position', ax = plt.subplot(j), title = nomi[tck])
        plt.title(nomi[tck])
        j += 1
plt.suptitle("OBV strategy", fontsize = 40)
# Save
#plt.savefig("img/backtesting/OBV_strategy2.png")

In [ ]:
# Confronto ritorni con strategie precedenti
plt.rcParams['figure.figsize'] = [18, 20]
j = 321
for sett in tcks:
    for tck in tcks[sett]:
        plt.subplot(j)
        plt.plot(strategy[tck]['Buy_and_hold'], label = 'Buy and hold')
        plt.plot(sma[tck]['Return'], label = 'SMA')
        plt.plot(ewm[tck]['Return'], label ='EWM')
        plt.plot(obv_rtn[tck]['Return'], label ='OBV')
        plt.legend()
        plt.title(nomi[tck])
        j += 1
plt.suptitle("Confronto ritorni con strategie precedenti", fontsize = 30)

# Save
#plt.savefig("img/backtesting/Rendimenti4.png")

In [ ]:
# CAPM 

# Calcolare il beta di ciascun titolo rispetto al mercato (indice S&P 500, ticker YahooFinance ^GSPC)
def init(freq):
    # Esempio di ticker, sostituisci con il tuo elenco
    tickers = ["AMZN", "GOOGL", "TSLA", "F", "PFE", "NVAX"]
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, start="2014-01-01", end="2024-01-01")
        if freq in ["MS", "W"]:
            df = df.resample(freq).mean(numeric_only=True)
        data[ticker] = df
    return data

# Calcola il beta di ciascun titolo rispetto al mercato (indice S&P 500, ticker YahooFinance ^GSPC)
nomi = {"^GSPC": "S&P 500"}

data_C = init("MS")
mrk = "^GSPC"
market_data = yf.download(mrk, start="2014-01-01", end="2024-01-01")
market_data = market_data.resample("MS").mean(numeric_only=True)
data_C[mrk] = market_data

# Calcolo delle matrici di covarianza
rtn_C = {} # rendimenti semplici
rtn_C["Complete"]  = pd.DataFrame()
for tck in data_C:
    rtn_C[tck] = pd.DataFrame(data_C[tck]["Adj Close"].pct_change())
    #rndS[tck] = np.log(adj[tck]/adj[tck].shift(1))
    rtn_C[tck] = rtn_C[tck].dropna()
    rtn_C[tck].columns = [tck]
    rtn_C["Complete"] = pd.concat([rtn_C["Complete"], rtn_C[tck]], axis = 1)

cov = rtn_C["Complete"].cov()

#Calcolo dei valori Beta
beta = cov.loc[mrk] / cov[mrk][mrk] # cov[mrk][mrk] --> Varianza dell'indice di mercato S&P 500
beta = beta.drop(mrk) #beta[mrk] è la varianza dell'indice di mercato S&P 500
beta

In [ ]:
# Funzioni utili al calcolo
def sigBeta (beta):
    if beta > 1:
        return "L'asset si muove nella stessa direzione del mercato ma la volatilità dell'asset è maggiore rispetto a quella del mercato"
    elif beta < 0:
        return "L'asset si muove nella direzione opposta del mercato"
    elif beta == 0:
        return "I movimenti del dell'asset e del mercato sono incorrelati"
    else: #0 < beta <= 1
        return "L'asset si muove nella stessa direzione del mercato ma la volatilità dell'asset può essere maggiore o minore rispetto a quella del mercato"
    
def maxSer (ser):
    max = ser.index[0]
    for ind in ser.index:
        if ser[ind] > ser[max]:
            max = ind
    return(max, ser[max])

def minSer (ser):
    min = ser.index[0]
    for ind in ser.index:
        if ser[ind] < ser[min]:
            min = ind
    return(min, ser[min])

In [ ]:
# Definizione dei dati di esempio
tcks = {
    "Tech": ["AMZN", "GOOGL"],
    "Auto": ["TSLA", "F"],
    "Med": ["PFE", "NVAX"]
}

nomi = {
    'AMZN': 'Amazon.com Inc.',
    'GOOGL': 'Alphabet Inc.',
    'TSLA': 'Tesla Inc.',
    'F': 'Ford Motor Company',
    'PFE': 'Pfizer Inc.',
    'NVAX': 'Novavax Inc.',
}

# Esempio di dati
dates = pd.date_range(start='2020-01-01', periods=100, freq='M')
adj = {
    'AMZN': pd.Series(range(100), index=dates),
    'GOOGL': pd.Series(range(100), index=dates),
    'TSLA': pd.Series(range(100), index=dates),
    'F': pd.Series(range(100), index=dates),
    'PFE': pd.Series(range(100), index=dates),
    'NVAX': pd.Series(range(100), index=dates),
}

for tck in beta.index:
    print("-" * 30 + " " + nomi[tck] + " " + "-" * 30)
    print("Beta : "  + str(beta[tck]))
    print(sigBeta(beta[tck]))

In [ ]:
mx = maxSer(beta)
mn = minSer(beta)
print("-" * 30 + " Massimo " + "-" * 30)
print("L'asset con valore beta maggiore è " + nomi[mx[0]] + " con un valore di beta: " + str(mx[1]))
print("-" * 30 + " Minimo " + "-" * 30)
print("L'asset con valore beta minore è " + nomi[mn[0]] + " con un valore di beta: " + str(mn[1]))

In [ ]:
# Utilizzo il beta per calcolare il rendimento atteso annuo del titolo
rf = 0.0138 #risk-free
rm = rtn_C[mrk].mean()[mrk] * 12 # Rendimento del mercato
ER = {} #Expected Return
for tck in beta.index:
    ER[tck] = rf + (beta[tck] * (rm-rf)) 

for tck in ER:
    print("-" * 60)
    print("Il rendimento atteso annuo di " + nomi[tck] + " è del " + str(round(ER[tck] * 100, 2)) + "%")

In [ ]:
# Calcolo l’esposizione di ciascun titolo ai fattori di rischio Fama-French
def init(freq):
    tickers = ["AMZN", "GOOGL", "TSLA", "F", "PFE", "NVAX"]
    data = {}
    for ticker in tickers:
        df = yf.download(ticker, start="2014-01-01", end="2024-01-01")
        if freq in ["M", "W"]:
            df = df.resample(freq).mean(numeric_only=True)
        data[ticker] = df
    return data

# Calcolo l’esposizione di ciascun titolo ai fattori di rischio Fama-French
factors = web.DataReader("F-F_Research_Data_Factors", "famafrench", start="2014-01-01", end="2024-01-01")[0].div(100)
factors.rename(columns={"Mkt-RF": "MKT"}, inplace=True)

data_FF = init("M")

for tck in data_FF:
    # Allineamento delle date: solo le date presenti in entrambi i dataframe
    aligned_dates = data_FF[tck].index.intersection(factors.index)
    data_FF[tck] = data_FF[tck].loc[aligned_dates]
    aligned_factors = factors.loc[aligned_dates]

    # Calcolo dei rendimenti dell'azione
    data_FF[tck] = pd.DataFrame(data_FF[tck]["Adj Close"].pct_change())
    data_FF[tck].columns = ["rtn"]

    # Unione con il dataframe dei fattori Fama-French 
    data_FF[tck] = pd.concat([aligned_factors, data_FF[tck]], axis=1)
    data_FF[tck] = data_FF[tck].dropna()

    # Calcoliamo i rendimenti in eccesso
    data_FF[tck]["exc"] = data_FF[tck]["rtn"] - data_FF[tck]["RF"]

    # Stampiamo i dati per verificare
    print(data_FF[tck].head())

model_formula = 'exc ~ MKT + SMB + HML'
params = {}
for tck in data_FF:
    print("-" * 30 + " " + nomi[tck] + " " + "-" * 30)
    ff_model = smf.ols(formula = model_formula, data = data_FF[tck]).fit()
    params[tck] = ff_model.params
    display(ff_model.summary())


In [ ]:
# COSTRUZIONE DEL PORTAFOGLIO
# Costruire il portafoglio ottimale in termini di media-varianza utilizzando i primi 108 mesi di dati, sia con metodo analitico sia con metodo di simulazione, utilizzando sia i rendimenti passati sia i rendimenti attesi costruiti nella parte 5
nomi["mv_sim"] = "Minimum Volatility ~ Simulazione"
nomi["msr_sim"] = "Maximum Sharpe Ratio ~ Simulazione"
nomi["eff"] = "Effettivo"
port = {}
n_month = 12
tot_month = 108
plt.rcParams['figure.figsize'] = [12, 7]

#Simulazione di portafogli con pesi casuali
n_port = 10 ** 6
n_assets = len(assets)
returns = rndS['Complete'][:tot_month].copy()
avg_returns = returns.mean() * n_month
cov_mat = returns.cov() * n_month
np.random.seed(42)
weights = np.random.random(size=(n_port, n_assets))
weights /=  np.sum(weights, axis=1)[:, np.newaxis]

#Calcoliamo i rendimenti dei portafogli
portf_rtns = np.dot(weights, avg_returns)
portf_vol = []

for i in range(0, len(weights)):
    portf_vol.append(np.sqrt(np.dot(weights[i].T, np.dot(cov_mat, weights[i]))))

portf_vol = np.array(portf_vol)  
portf_sharpe_ratio = portf_rtns / portf_vol
portf_results_df = pd.DataFrame({'returns': portf_rtns,
                                 'volatility': portf_vol,
                                 'sharpe_ratio': portf_sharpe_ratio})

#Creazione della frontiera efficiente
n_points = 1000
portf_vol_ef = []
indices_to_skip = []

portf_rtns_ef = np.linspace(portf_results_df.returns.min(), 
                            portf_results_df.returns.max(), 
                           n_points)
portf_rtns_ef = np.round(portf_rtns_ef, 2)    
portf_rtns = np.round(portf_rtns, 2)

for point_index in range(n_points):
    if portf_rtns_ef[point_index] not in portf_rtns:
        indices_to_skip.append(point_index)
        continue
    matched_ind = np.where(portf_rtns == portf_rtns_ef[point_index])
    portf_vol_ef.append(np.min(portf_vol[matched_ind]))
    
portf_rtns_ef = np.delete(portf_rtns_ef, indices_to_skip)

MARKS = ['o', 'X', 's', '*', 'D', 'H' ]
fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter', x='volatility', 
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', 
                      ax=ax)
ax.set(xlabel='Volatility', 
       ylabel='Expected Returns', 
       title='Efficient Frontier')
ax.plot(portf_vol_ef, portf_rtns_ef, 'b--')
for asset_index in range(n_assets):
    ax.scatter(x=np.sqrt(cov_mat.iloc[asset_index, asset_index]), 
                y=avg_returns[asset_index], 
                marker=MARKS[asset_index], 
                s=200, 
                color='black',
                label=tickers[asset_index])
ax.legend()
plt.tight_layout()
plt.show()

#Ricerca del portafoglio con l'indice di Sharpe più alto
max_sharpe_ind = np.argmax(portf_results_df.sharpe_ratio)
max_sharpe_portf = portf_results_df.loc[max_sharpe_ind]

port["msr_sim"] = {}
port["msr_sim"]["Return"] = max_sharpe_portf[0]
port["msr_sim"]["Volatility"] = max_sharpe_portf[1]
port["msr_sim"]["Sharpe Ratio"] = max_sharpe_portf[2]

#Ricerca del portafoglio con volatilità minore
min_vol_ind = np.argmin(portf_results_df.volatility)
min_vol_portf = portf_results_df.loc[min_vol_ind]
port["mv_sim"] = {}
port["mv_sim"]["Return"] = min_vol_portf[0]
port["mv_sim"]["Volatility"] = min_vol_portf[1]
port["mv_sim"]["Sharpe Ratio"] = min_vol_portf[2]

#STAMPE 
print('-' * 20 + ' Maximum Sharpe Ratio portfolio ' + '-' * 20)
print('Performance')
for index, value in max_sharpe_portf.items():
    print(f'{index}: {100 * value:.2f}% ', end="", flush=True)
print('\nWeights')
port["msr_sim"]["weights"] = OrderedDict()
for x, y in zip(tickers, weights[np.argmax(portf_results_df.sharpe_ratio)]):
    print(f'{x}: {100*y:.2f}% ', end="", flush=True)
    port["msr_sim"]["weights"][x] = y
print()

print('-' * 20 + ' Minimum Volatility portfolio ' + '-' * 20)
print('Performance')
for index, value in min_vol_portf.items():
    print(f'{index}: {100 * value:.2f}% ', end="", flush=True)
port["mv_sim"]["weights"] = OrderedDict()
print('\nWeights')
for x, y in zip(tickers, weights[np.argmin(portf_results_df.volatility)]):
    print(f'{x}: {100*y:.2f}% ', end="", flush=True)
    port["mv_sim"]["weights"][x] = y

#GRAFICO
fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter', x='volatility', 
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', 
                      ax=ax)

# Max Sharpe Ratio
ax.scatter(x = port["msr_sim"]["Volatility"], 
           y = port["msr_sim"]["Return"], 
           c='black', marker='*', 
           s=200, label='Max Sharpe Ratio')

# Min Volatility
ax.scatter(x = port["mv_sim"]["Volatility"], 
           y = port["mv_sim"]["Return"], 
           c='black', marker='P', 
           s=200, label='Minimum Volatility')

ax.set(xlabel='Volatility', ylabel='Expected Returns', 
       title='Efficient Frontier')
ax.legend()
plt.suptitle("Simulazione", fontsize = 30)
plt.tight_layout()

# Save
#plt.savefig("img/port/Simulazione.png")

In [ ]:
# PORTAFOLGIO EFFETTIVO

#Pesi
port["eff"] = {}
port["eff"]["weights"] = OrderedDict()
for tck in tickers:
    port["eff"]["weights"][tck] = 1 / len(tickers)

#Return 
for pt in port:
    port[pt]["rtn"] = rndS["Complete"][:108].copy()
    for tck in tickers:
        port[pt]["rtn"][tck] *= port[pt]["weights"][tck]
    port[pt]["rtn"] = pd.DataFrame(port[pt]["rtn"].sum(axis = 1))
    port[pt]["rtn"].rename(columns = {0 : pt}, inplace = True)

port["eff"]["Return"] = port["eff"]["rtn"].mean()["eff"] * n_month

# Volatility
port["eff"]["Volatility"] = port["eff"]["rtn"].std()["eff"] * np.sqrt(n_month)

# Sharpe Ratio
port["eff"]["Sharpe Ratio"] = port["eff"]["Return"] / port["eff"]["Volatility"]

# STAMPE
def portfolioPrint (portfolio):
    print('Performance')
    performance = ['Return', 'Volatility', 'Sharpe Ratio']
    for per in performance:
        print(per + ": " + str(round(portfolio[per] * 100, 2)) + "%", end="\t" , flush = True)
    print('\nWeights')
    for wg in portfolio["weights"]:
        print(wg + ": " + str(round(portfolio["weights"][wg] * 100, 2)) + "%", end="\t" , flush = True)
    print()
    
print('-' * 20 + ' ' + nomi["eff"] + ' ' + '-' * 20)
portfolioPrint(port["eff"])

# Stampa di tutti i portafogli
for pt in port:
    print('-' * 20 + ' ' + nomi[pt] + ' ' + '-' * 20)
    portfolioPrint(port[pt])

# Grafico
plt.rcParams['figure.figsize'] = [12, 7]
fig, ax = plt.subplots()
#Frontiera Efficiente
portf_results_df.plot(kind='scatter', x='volatility', 
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', 
                      ax=ax)

# Portafoglio effettivo
ax.scatter(x = port["eff"]["Volatility"], 
           y = port["eff"]["Return"], 
           c = 'black', marker='*', 
           s=200, label = nomi["eff"])


ax.set(xlabel='Volatility', 
       ylabel='Expected Returns', 
       title='Efficient Frontier')
plt.suptitle("Effettivo", fontsize = 30)
ax.legend()
plt.tight_layout()
# Save
#plt.savefig("img/port/effettivo.png")

In [ ]:
mk = ['o', 'X', '*', 'D', 'P']
cl = ['black','white','white','black','black']
for pt in port:
    port[pt]["mk"] = mk.pop()
    port[pt]["cl"] = cl.pop()

plt.rcParams['figure.figsize'] = [12, 7]
fig, ax = plt.subplots()
#Frontiera Efficiente
portf_results_df.plot(kind='scatter', x='volatility', 
                      y='returns', c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', 
                      ax=ax)

# Portafogli 
for pt in port:
       ax.scatter(x = port[pt]["Volatility"], 
           y = port[pt]["Return"], 
           c = port[pt]["cl"], marker = port[pt]["mk"], 
           s=200, label = nomi[pt])


ax.set(xlabel='Volatility', 
       ylabel='Expected Returns', 
       title='Efficient Frontier')
plt.suptitle("Tutti i portafogli", fontsize = 30)
ax.legend()
plt.tight_layout()
# Save
#plt.savefig("img/port/all.png")

In [ ]:
# Calcolare il beta del portafoglio rispetto al mercato

# Uso l'indice di mercato S&P 500 come benchmark per calcolare poi il beta
mrkt = yf.download("^GSPC", start = day[0], end = day[1])
mrkt = mrkt.resample("M").mean(numeric_only=True)
rM = mrkt["Adj Close"].pct_change().dropna()

# Creo un dataframe contenete tutti i rendimenti
rS = pd.DataFrame()
for pt in port:
    rS= pd.concat([rS, port[pt]["rtn"]], axis = 1)

#Creo una nuova variabile che contiene il dataframe calcolato precendentemente con l'aggiunta dei rendimenti del mercato
rSM = pd.concat([rS, rM[:tot_month]], axis = 1)
rSM.rename(columns = {"Adj Close" : "MKT"}, inplace = True)
cov = rSM.cov()
beta = cov.loc["MKT"] / cov["MKT"]["MKT"]
beta = beta.drop("MKT")

# Stampe risultati
for tck in beta.index:
    print("-" * 30 + " " + nomi[tck] + " " + "-" * 30)
    print("Beta : "  + str(beta[tck]))
    print(sigBeta(beta[tck]))

mx = maxSer(beta)
mn = minSer(beta)
print("-" * 30 + " Massimo " + "-" * 30)
print("Il portafoglio con valore beta maggiore è " + nomi[mx[0]] + " con un valore di beta: " + str(mx[1]))
print("-" * 30 + " Minimo " + "-" * 30)
print("Il portafoglio con valore beta minore è " + nomi[mn[0]] + " con un valore di beta: " + str(mn[1]))

In [ ]:
# Confrontare il rendimento del portafoglio ottimale con quello effettivo
# Per “portafoglio effettivo” si intende un portafoglio composto dai sei titoli oggetto di analisi con peso uguale fra di loro

rC = (rS + 1).cumprod()
rC.plot()

for pt in port:
    port[pt]["rtnC"] = (port[pt]["rtn"] + 1).cumprod()
colors = ["red", "purple", "orange", "blue", "black"]

for pt in port:
    port[pt]["color"] = colors.pop()
plt.rcParams['figure.figsize'] = [12, 7]
fig, ax = plt.subplots()

for pt in port:
    ax.plot(port[pt]["rtnC"], label = nomi[pt], color = port[pt]["color"], linewidth = 0.7)
plt.legend()
plt.title("Andamento rendimenti composti dei portafogli")

# Save
#plt.savefig("img/port/rendCom.png")